# Kickstarter Project

### In this project we analyze which features affects on the success of Kickstarter projects. We perform analysis using ML algorithms – Random Forest Classifier and SVM.

# Part 1: Data Preperation

### In this part we prepare the dataset - extract json columns to new columns, Removed irrelevant columns, remove rows that included null cells. Then, apply string indexer for string columns.

## Imports & Configurations

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -O ./spark-3.3.1-bin-hadoop3.tgz  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar zxvf ./spark-3.3.1-bin-hadoop3.tgz
!pip install findspark
!pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

import findspark
import pyspark
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer, OneHotEncoder

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "2g")\
    .config("spark.executor.memory", "2g") \
    .config('spark.executor.cores', '3') \
    .config('spark.cores.max', '3') \
    .appName("prepare").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 10)
spark.conf.set("spark.default.parallelism", 10)

## Read the Data

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/BDP Final Project/Notebooks/Data/alldata.csv",
                    inferSchema=True,
                    header=True,
                    escape='"')

In [ ]:
df.show()

#Part 1.1: Data Cleaning

## Prepare Schema

In [ ]:
df.printSchema()

In [ ]:
df.select('category').show(5,False)

In [ ]:
category_schema = T.StructType([
    T.StructField("id", T.LongType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("slug", T.StringType(), True),
    T.StructField("position", T.StringType(), True),
    T.StructField("parent_id", T.LongType(), True),
    T.StructField("parent_name", T.StringType(), True),
    T.StructField("color", T.LongType(), True),
    T.StructField("urls", T.StringType(), True)
    ]
)
df = df.withColumn("category", F.from_json(F.col("category"), category_schema))


In [ ]:
df.select('creator').show(5,False)

In [ ]:
creator_schema = T.StructType([
    T.StructField("id", T.LongType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("slug", T.StringType(), True),
    T.StructField("is_registered", T.StringType(), True),
    T.StructField("is_email_verified", T.StringType(), True),
    T.StructField("chosen_currency", T.StringType(), True),
    T.StructField("is_superbacker", T.StringType(), True),
    ]
)
df = df.withColumn("creator", F.from_json(F.col("creator"), creator_schema))


In [ ]:
df.select('location').show(5,False)

In [ ]:
location_schema = T.StructType([
    T.StructField("id", T.LongType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("slug", T.StringType(), True),
    T.StructField("short_name", T.StringType(), True),
    T.StructField("displayable_name", T.StringType(), True),
    T.StructField("localized_name", T.StringType(), True),
    T.StructField("country", T.StringType(), True),
    T.StructField("state", T.StringType(), True),
    T.StructField("type", T.StringType(), True),
    T.StructField("is_root", T.StringType(), True),
    T.StructField("expanded_country", T.StringType(), True),
    ]
)
df = df.withColumn("location", F.from_json(F.col("location"), location_schema))


In [ ]:
df.select('profile').show(5,False)

In [ ]:
profile_schema = T.StructType([
    T.StructField("id", T.LongType(), True),
    T.StructField("project_id", T.LongType(), True),
    T.StructField("state", T.StringType(), True),
    T.StructField("state_changed_at", T.StringType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("blurb", T.StringType(), True),
    T.StructField("background_color", T.StringType(), True),
    T.StructField("text_color", T.StringType(), True),
    T.StructField("link_background_color", T.StringType(), True),
    T.StructField("link_text_color", T.StringType(), True),
    T.StructField("link_text", T.StringType(), True),
    T.StructField("link_url", T.StringType(), True),
    T.StructField("show_feature_image", T.StringType(), True),
    T.StructField("background_image_opacity", T.DoubleType(), True),
    T.StructField("should_show_feature_image_section", T.BooleanType(), True),
    ]
)
df = df.withColumn("profile", F.from_json(F.col("profile"), profile_schema))


In [ ]:
df.printSchema()

In [ ]:
df = df.select(
    F.col('id').cast('long').alias('id'),
    F.col('name').cast('string').alias('name'),
    F.col('blurb').cast('string').alias('blurb'),
    F.col('slug').cast('string').alias('slug'),
    F.col('backers_count').cast('long').alias('backers_count'),

    F.col('category.id').cast('long').alias('category_id'),
    F.col('category.name').cast('string').alias('category_name'),
    F.col('category.slug').cast('string').alias('category_slug'),
    F.col('category.position').cast('string').alias('category_position'),
    F.col('category.parent_id').cast('long').alias('category_parent_id'),
    F.col('category.parent_name').cast('string').alias('category_parent_name'),
    F.col('category.color').cast('long').alias('category_color'),

    F.col('converted_pledged_amount').cast('long').alias('converted_pledged_amount'),
    F.col('currency').cast('string').alias('currency'),
    F.col('currency_symbol').cast('string').alias('currency_symbol'),
    F.col('currency_trailing_code').cast('boolean').alias('currency_trailing_code'),
    F.col('current_currency').cast('string').alias('current_currency'),
    F.col('static_usd_rate').cast('double').alias('static_usd_rate'),
    F.col('usd_pledged').cast('double').alias('usd_pledged'),
    F.col('usd_type').cast('string').alias('usd_type'),
    F.col('fx_rate').cast('double').alias('fx_rate'),

    F.col('pledged').cast('double').alias('pledged'),
    F.col('goal').cast('double').alias('goal'),

    F.col('created_at').cast('long').alias('created_at_epoch'),
    F.from_unixtime(F.col("created_at"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('created_at_value'),
    F.col('deadline').cast('long').alias('deadline_epoch'),
    F.from_unixtime(F.col("deadline"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('deadline_value'),
    F.col('launched_at').cast('long').alias('launched_at_epoch'),
    F.from_unixtime(F.col("launched_at"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('launched_at_value'),
    F.col('state_changed_at').cast('long').alias('state_changed_at_epoch'),
    F.from_unixtime(F.col("state_changed_at"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('state_changed_at_value'),
    F.col('last_update_published_at').cast('long').alias('last_update_published_at_epoch'),
    F.from_unixtime(F.col("last_update_published_at"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('last_update_published_at_value'),

    F.col('creator.id').cast('long').alias('creator_id'),
    F.col('creator.name').cast('string').alias('creator_name'),
    F.col('creator.slug').cast('string').alias('creator_slug'),
    F.col('creator.is_registered').cast('string').alias('creator_is_registered'),
    F.col('creator.is_email_verified').cast('string').alias('creator_is_email_verified'),
    F.col('creator.chosen_currency').cast('string').alias('creator_chosen_currency'),
    F.col('creator.is_superbacker').cast('string').alias('creator_is_superbacker'),

    F.col('disable_communication').cast('boolean').alias('disable_communication'),
    F.col('friends').cast('string').alias('friends'),
    F.col('is_backing').cast('boolean').alias('is_backing'),
    F.col('is_starrable').cast('boolean').alias('is_starrable'),
    F.col('is_starred').cast('boolean').alias('is_starred'),
    F.col('permissions').cast('string').alias('permissions'),
    F.col('spotlight').cast('boolean').alias('spotlight'),
    F.col('staff_pick').cast('boolean').alias('staff_pick'),
    F.col('unread_messages_count').cast('long').alias('unread_messages_count'),
    F.col('unseen_activity_count').cast('long').alias('unseen_activity_count'),

    F.col('country').cast('string').alias('country'),
    F.col('country_displayable_name').cast('string').alias('country_displayable_name'),
    F.col('location.id').cast('long').alias('location_id'),
    F.col('location.name').cast('string').alias('location_name'),
    F.col('location.slug').cast('string').alias('location_slug'),
    F.col('location.short_name').cast('string').alias('location_short_name'),
    F.col('location.displayable_name').cast('string').alias('location_displayable_name'),
    F.col('location.localized_name').cast('string').alias('location_localized_name'),
    F.col('location.country').cast('string').alias('location_country'),
    F.col('location.state').cast('string').alias('location_state'),
    F.col('location.type').cast('string').alias('location_type'),
    F.col('location.is_root').cast('boolean').alias('location_is_root'),
    F.col('location.expanded_country').cast('boolean').alias('location_expanded_country'),
   
    F.col('profile.id').cast('long').alias('profile_id'),
    F.col('profile.project_id').cast('long').alias('profile_project_id'),
    F.col('profile.state').cast('string').alias('profile_state'),
    F.col('profile.state_changed_at').cast('string').alias('profile_state_changed_at_epoch'),
    F.from_unixtime(F.col("profile.state_changed_at"),"MM-dd-yyyy HH:mm:ss").cast('string').alias('profile_state_changed_at_value'),
    F.col('profile.name').cast('string').alias('profile_name'),
    F.col('profile.blurb').cast('string').alias('profile_blurb'),
    F.col('profile.background_color').cast('string').alias('profile_background_color'),
    F.col('profile.text_color').cast('string').alias('profile_text_color'),
    F.col('profile.link_background_color').cast('string').alias('profile_link_background_color'),
    F.col('profile.link_text_color').cast('string').alias('profile_link_text_color'),
    F.col('profile.link_text').cast('string').alias('profile_link_text'),
    F.col('profile.link_url').cast('string').alias('profile_link_url'),
    F.col('profile.show_feature_image').cast('string').alias('profile_show_feature_image'),
    F.col('profile.background_image_opacity').cast('string').alias('profile_background_image_opacity'),
    F.col('profile.should_show_feature_image_section').cast('string').alias('profile_should_show_feature_image_section'),
    
    F.col('state').cast('string').alias('state'),
)

In [ ]:
df.printSchema()

In [ ]:
df.show()

##Remove not relvant columns

In [ ]:
columns_count = len(df.columns)
columns_count

In [ ]:
rows_count = df.count()
rows_count

Remove columns that contains multiple null values:

In [ ]:
null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()  

In [ ]:
columns_to_drop = [k for k, v in null_counts.items() if v >= rows_count * 0.5]
columns_to_drop

In [ ]:
df = df.drop(*columns_to_drop)

Remove irrelevant columns:

In [ ]:
columns_to_drop = ['category_position','category_color','category_slug','slug',
  'converted_pledged_amount','currency_symbol','currency_trailing_code','static_usd_rate','current_currency',
  'usd_pledged','usd_type','fx_rate',
  'creator_slug','creator_id','creator_name',
  'country_displayable_name','location_name','location_slug','location_short_name','location_displayable_name',
  'location_localized_name','location_id',
  'country','profile_id','profile_project_id','profile_show_feature_image','profile_background_image_opacity',
  'profile_should_show_feature_image_section'
]

df = df.drop(*columns_to_drop)

In [ ]:
df.show()

## Remove corrupted and irrelevant rows

In [ ]:
df = df.where(
    F.col('state').isin(['successful','failed'])
)

In [ ]:
df = df.where(
    ((F.col('name').startswith('{') == False) | (F.col('name') == None)) &
    ((F.col('blurb').startswith('{') == False) | (F.col('blurb') == None)) &
    ((F.col('slug').startswith('{') == False) | (F.col('slug') == None)) &
    ((F.col('usd_type').startswith('{') == False) | (F.col('usd_type') == None)) &
    ((F.col('country').startswith('{') == False) | (F.col('country') == None)) &
    ((F.length(F.col('currency_symbol')) == 1) | (F.col('currency_symbol') == None) )
)

In [ ]:
df = df\
.withColumn('blurb', F.regexp_replace(F.col('blurb'), '"', ''))\
.withColumn('name', F.regexp_replace(F.col('name'), '"', ''))

In [ ]:
df = df.na.drop(how="any")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.show()

## Data Encoding

Encode string values:

In [ ]:
df = df.withColumn("state_index", F.when(F.col('state') == 'successful', 1).otherwise(0))

In [ ]:
df_types = df.dtypes
string_columns_not_to_encode = \
  ['name', 'blurb', 'deadline_value', 'created_at_value', 'launched_at_value', 'state_changed_at_value', 'profile_state_changed_at_value', 'state']
string_columns = []

for column_name, column_type in df_types:
  if column_type == 'string' and column_name not in string_columns_not_to_encode:
    string_columns.append(column_name)  
                
indexer = StringIndexer(
    inputCols=string_columns, 
    outputCols=["{}_index".format(c) for c in string_columns],
    handleInvalid='keep'
)

df = indexer.fit(df).transform(df)

Encoding boolean values:

In [ ]:
for column_name, column_type in df_types:
  if column_type == 'boolean':
    df = df.withColumn(f"{column_name}_index", F.col(column_name).cast("integer"))

In [ ]:
df.show()

## Save CSV

In [ ]:
df.write.csv("/content/drive/MyDrive/BDP Final Project/Notebooks/Data/prepared_data.csv", 
             header=True, mode ="overwrite")

# Part 1.2: Data Visualization

In [ ]:
def plot_counter_pie(df, column):
    """Plot counter pie of given column of the dataframe."""
    data = df.groupBy(column).count()
    sizes = [val['count'] for val in data.select('count').collect()]
    lables = [val[column] for val in data.select(column).collect()]

    theme = plt.get_cmap('GnBu')
    
    fig, ax = plt.subplots(figsize=(10, 9))
    ax.set_prop_cycle("color", [theme(1. * i / len(sizes)) for i in range(len(sizes))])
    ax.pie(sizes,
           labels=lables,
           autopct='%1.1f%%',
           textprops={'size': 15}
          )
    ax.set_title("percent frequency values of " + str(column), fontsize=22)
    
    plt.legend(
        loc='upper left',
        labels=['%s: %1.1f%%' % (
            l, (float(s) / sum(sizes)) * 100) for l, s in zip(lables, sizes)],
        prop={'size': 15},
        bbox_to_anchor=(-0.3, 0.8),
        bbox_transform=fig.transFigure
    )

    plt.show()

def plot_column_state_compare(df, column, hue='state', number_of_counts=10):
    """Plot state compare of given column of the dataframe."""
    data = df.select([column, hue]).toPandas()

    fig, ax = plt.subplots(1,1, figsize=(12,10))
    plt.subplot(212)

    ax = sns.countplot(x=column, hue=hue, data=data, palette='GnBu',
                       order=data[column].value_counts().iloc[:number_of_counts].index)
    
    ax.set_title("Projects count by " + str(column), fontsize=22)
    ax.set_xlabel(str(column), fontsize=17)
    ax.set_ylabel("Count", fontsize=17),
    ax.legend(loc='upper right')
    plt.subplots_adjust(hspace = 0.4)
    
    plt.show()

Plotting the successful and failed frequency:

In [ ]:
plot_counter_pie(df, 'state')

Plotting the parent category frequency:

In [ ]:
plot_counter_pie(df, 'category_parent_name')

Plotting compare successful and failed by State:

In [ ]:
plot_column_state_compare(df, 'location_state')

Plotting compare successful and failed by Country:

In [ ]:
plot_column_state_compare(df, 'location_country')

Plotting compare successful and failed by Location Type:

In [ ]:
plot_column_state_compare(df, 'location_type')

Bar Plot of mean backers by parent category:

In [ ]:
data = df.select(['category_parent_name', 'backers_count']).toPandas()
data = data.groupby('category_parent_name').mean().reset_index()

fig, ax = plt.subplots(1,1, figsize=(12,6))
sns.barplot(x="category_parent_name", y="backers_count", data=data, palette="GnBu", ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.show()

Plotting compare succussful or failed by goal:
* Used log1p to get more accuary plot

In [ ]:
data = df.select(['goal', 'backers_count', 'state']).toPandas()

In [ ]:
failed = np.log1p(data[data.state == 'failed'].goal)
successful = np.log1p(data[data.state == 'successful'].goal)

plt.figure(figsize=(12,6))
sns.distplot(failed, bins=30, norm_hist=True, color = 'c')
sns.distplot(successful, bins=30, norm_hist=True, color = 'b')
plt.legend(title='State', loc='upper left', labels=['failed', 'successful'])

plt.show()

Plotting compare succussful or failed by backers:

In [ ]:
failed = np.log1p(data[data.state == 'failed'].backers_count)
successful = np.log1p(data[data.state == 'successful'].backers_count)

plt.figure(figsize=(12,6))
sns.distplot(failed, bins=30, norm_hist=True, color = 'c')
sns.distplot(successful, bins=30, norm_hist=True, color = 'b')
plt.legend(title='State', loc='upper left', labels=['failed', 'successful'])

plt.show()

In [ ]:
spark.stop()

#Part 1.3: Conclusion


### * The number of backers has a big impact on success.
### * As much as the goal is lower, the more chances it'll be a successful.
### * In the distribution of the goal variable, we can observe goal is highly skewed to the right.
### * Games, Design and Technology categories have the most high backers mean.
### * Most of the projects are from US.